In [ ]:
import numpy as np
import pandas as pd
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
train_path = './covid/train/'  
test_path = './covid/test/' 

#데이터 불러오고 필요없는 칼럼 삭제
#train 데이터
train_df = pd.read_csv('./covid/train.txt', sep=" ", header=None)
train_df.columns=['patient_id', 'filename', 'class', 'data_source']
train_df=train_df.drop(['patient_id', 'data_source'], axis=1 )

#test 데이터
test_df = pd.read_csv('./covid/test.txt', sep=" ", header=None)
test_df.columns=['patient_id', 'filename', 'class', 'data_source' ]
test_df=test_df.drop(['patient_id', 'data_source'], axis=1 )
train_df.head()

#class coluum에 카테고리값 삽입
negative  = train_df[train_df['class']=='negative']   
positive = train_df[train_df['class']=='positive']  
from sklearn.utils import resample

#이 데이터는 negative데이터가 비중이 커서, positive data 개수(2158)에 맞춘다.
df_downsampled = resample(negative, replace = True, n_samples = 2158) 
train_df = pd.concat([positive, df_downsampled])

from sklearn.utils import shuffle
train_df = shuffle(train_df)

#positive, negative 개수 같은지 확인
train_df['class'].value_counts()

#train_test_split함수 사용하여 train데이터와 valid데이터 나눔
train_df, valid_df = train_test_split(train_df, train_size=0.9, random_state=0)

#이미지 전처리 
#(https://keras.io/ko/preprocessing/image/ 에서 도움 얻음)
#rescale: 크기 재조절 인수
#rotation_range: 무작위 회전의 각도 범위
#width_shift_range, height_shift_range: 1이하인 경우 부동소수점 -> 전체 가로넓이에서의 비율, 전체 세로높이에서의 비율
# shear_range, zoom_range : 층밀리기의 강도, 무작위 줌의 범위
#horizontal_flip,  vertical_flip: input을 무작위로 가로, 세로로 뒤집음.

train_datagenerator = ImageDataGenerator(rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True, vertical_flip =True)

test_datagenerator = ImageDataGenerator(rescale = 1./255.)

train_gen = train_datagenerator.flow_from_dataframe(dataframe = train_df, directory=train_path, x_col='filename',  y_col='class', target_size=(224,224), batch_size=64, class_mode='binary')
valid_gen = test_datagenerator.flow_from_dataframe(dataframe = valid_df, directory=train_path, x_col='filename', y_col='class', target_size=(224,224), batch_size=64, class_mode='binary')

test_gen = test_datagenerator.flow_from_dataframe(dataframe = test_df, directory=test_path, x_col='filename', y_col='class', target_size=(224,224), batch_size=64, class_mode='binary')

#ResNet50V2 모델 로드
base_model = tf.keras.applications.ResNet50V2(weights='imagenet', input_shape = (224,224,3),
                                                     include_top=False)
#레이어와 이미 학습된 가중치만 사용
for layer in base_model.layers:
    layer.trainable = False

#layer 한번더 쌓기
#GlobalAveragePooling2D()은 강의자료의 Flatten()과 비슷한 역할을 함.

model = tf.keras.Sequential([
    base_model, 
    tf.keras.layers.GlobalAveragePooling2D(), 
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.BatchNormalization(), 
    tf.keras.layers.Dropout(0.2), 

#activation은 sigmoid와 softmax을 비교하였는데, softmax는 다중클래스 분류에 유용하여 단일 클래스 분류에 유용한 sigmoid보다 학습속도가 오래 걸리고 정확도도 떨어져서 sigmoid로 정하였다.

    tf.keras.layers.Dense(1, activation='sigmoid')
])

#케라스 콜백 사용하여 모델의 훈련 과정 제어하기
callbacks = [
    #체크포인트 저장 : 훈련하는 동안 어떤 지점에서 모델의 현재 가중치 저장
    tf.keras.callbacks.ModelCheckpoint("covid_classifier_model.h5", save_best_only=True, verbose = 0),
    #조기 종료: 검증 손실이 더 이상 향상되지 않을 때에 훈련 중지
    tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss', verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)
]

#모델 학습
model.compile(optimizer = keras.optimizers.Adam(learning_rate=0.001),
              loss = 'binary_crossentropy',
              metrics=['accuracy'])
history = model.fit(train_gen, 
                    validation_data=valid_gen, epochs=20, 
                    callbacks=[callbacks])


#test데이터로 모델 평가
model.load_weights('./covid_classifier_model.h5')
model.evaluate(test_gen)


In [ ]:
#ResNet50V2 모델 로드
base_model = tf.keras.applications.ResNet50V2(weights='imagenet', input_shape = (224,224,3),
                                                     include_top=False)
#레이어와 이미 학습된 가중치만 사용
for layer in base_model.layers:
    layer.trainable = False

#layer 한번더 쌓기
#GlobalAveragePooling2D()은 강의자료의 Flatten()과 비슷한 역할을 함.

model = tf.keras.Sequential([
    base_model, 
    tf.keras.layers.GlobalAveragePooling2D(), 
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.BatchNormalization(), 
    tf.keras.layers.Dropout(0.2), 

#activation은 sigmoid와 softmax을 비교하였는데, softmax는 다중클래스 분류에 유용하여 단일 클래스 분류에 유용한 sigmoid보다 학습속도가 오래 걸리고 정확도도 떨어져서 sigmoid로 정하였다.

    tf.keras.layers.Dense(1, activation='sigmoid')
])

#케라스 콜백 사용하여 모델의 훈련 과정 제어하기
callbacks = [
    #체크포인트 저장 : 훈련하는 동안 어떤 지점에서 모델의 현재 가중치 저장
    tf.keras.callbacks.ModelCheckpoint("covid_classifier_model.h5", save_best_only=True, verbose = 0),
    #조기 종료: 검증 손실이 더 이상 향상되지 않을 때에 훈련 중지
    tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss', verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)
]

#모델 학습
model.compile(optimizer = keras.optimizers.Adam(learning_rate=0.001),
              loss = 'binary_crossentropy',
              metrics=['accuracy'])
history = model.fit(train_gen, 
                    validation_data=valid_gen, epochs=20, 
      
#test데이터로 모델 평가
model.load_weights('./covid_classifier_model.h5')
model.evaluate(test_gen)
              callbacks=[callbacks])


In [ ]:
#다시 학습하기
import tensorflow as tf
base_model = tf.keras.applications.ResNet50V2(input_shape = (224,224,3),
                                                     include_top=False)

model = tf.keras.Sequential([
    base_model, 
    tf.keras.layers.GlobalAveragePooling2D(), 
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.BatchNormalization(), 
    tf.keras.layers.Dropout(0.2), 
    tf.keras.layers.Dense(1, activation='sigmoid')
])


#케라스 콜백 사용하여 모델의 훈련 과정 제어하기
callbacks = [
    tf.keras.callbacks.ModelCheckpoint("covid_classifier_model.h5", save_best_only=True, verbose = 0),
    tf.keras.callbacks.EarlyStopping(patience=1, monitor='val_loss', verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=1, verbose=1)
]
#모델 학습
model.compile(optimizer = keras.optimizers.Adam(learning_rate=0.001),
              loss = 'binary_crossentropy',
              metrics=['accuracy'])
history = model.fit(train_gen, validation_data=valid_gen, epochs=30, 
                    callbacks=callbacks,verbose=1)

#test데이터로 모델 평가
model.load_weights('./covid_classifier_model.h5')
model.evaluate(test_gen)

#예측
pred = (model.predict(test_gen)>0.5).astype("int32")
print(pred[0:10])


In [ ]:
#다시
import tensorflow as tf
base_model = tf.keras.applications.ResNet50V2(input_shape = (224,224,3),
                                                     include_top=False)
model = tf.keras.Sequential([
    base_model, 
    tf.keras.layers.GlobalAveragePooling2D(), 
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.BatchNormalization(), 
    tf.keras.layers.Dropout(0.3), 
    tf.keras.layers.Dense(1, activation='sigmoid')
])

callbacks = [
    tf.keras.callbacks.ModelCheckpoint("covid_classifier_model.h5", save_best_only=True, verbose = 0),
    tf.keras.callbacks.EarlyStopping(patience=1, monitor='val_loss', verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=1, verbose=1)
]

#모델 학습
model.compile(optimizer = keras.optimizers.Adam(learning_rate=0.01), loss = 'binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_gen, validation_data=valid_gen, epochs=30, callbacks=callbacks)
